## Avaliando [Crash Course](https://mxnet.apache.org/versions/master/api/python/docs/tutorials/getting-started/crash-course/4-components.html)


In [1]:
from mxnet import np,npx,gluon
import mxnet as mx
from mxnet.gluon import nn
npx.set_np()

device=mx.cpu()

In [2]:
net=nn.Sequential()

net.add(nn.Dense(5,in_units=3,activation='relu'),
        nn.Dense(25,activation='relu'),
        nn.Dense(2)
)

net

Sequential(
  (0): Dense(3 -> 5, Activation(relu))
  (1): Dense(-1 -> 25, Activation(relu))
  (2): Dense(-1 -> 2, linear)
)

In [3]:
net.initialize()
params=net.collect_params()

for key,value in params.items():
    print(key,value)

0.weight Parameter (shape=(5, 3), dtype=float32)
0.bias Parameter (shape=(5,), dtype=float32)
1.weight Parameter (shape=(25, -1), dtype=float32)
1.bias Parameter (shape=(25,), dtype=float32)
2.weight Parameter (shape=(2, -1), dtype=float32)
2.bias Parameter (shape=(2,), dtype=float32)


[11:00:05] /home/mertins/Desenvolvimento/C/Terceiros/mxnet/src/storage/storage.cc:202: Using Pooled (Naive) StorageManager for CPU


In [4]:
x=np.random.uniform(-1,1,(10,3))
net(x) #Forward computation

params=net.collect_params()
for key,value in params.items():
    print(key,value)

0.weight Parameter (shape=(5, 3), dtype=float32)
0.bias Parameter (shape=(5,), dtype=float32)
1.weight Parameter (shape=(25, 5), dtype=float32)
1.bias Parameter (shape=(25,), dtype=float32)
2.weight Parameter (shape=(2, 25), dtype=float32)
2.bias Parameter (shape=(2,), dtype=float32)


#### pesos inicializados na primeira camada

In [5]:
print(net[0].weight.data()[0])
print(net[0].weight.data()[1])
print(net[0].weight.data()[2])
print(net[0].weight.data()[3])
print(net[0].weight.data()[4])

[0.0068339  0.01299825 0.0301265 ]
[0.04819721 0.01438687 0.05011239]
[ 0.00628365  0.04861524 -0.01068833]
[ 0.01729892  0.02042518 -0.01618656]
[-0.00873779 -0.02834515  0.05484822]


### Build-in Initialization

In [6]:
from mxnet import init

net.initialize(init=init.Constant(3),device=device,force_reinit=True)
print(net[0].weight.data()[0])
print(net[0].weight.data()[1])
print(net[0].weight.data()[2])
print(net[0].weight.data()[3])
print(net[0].weight.data()[4])

[3. 3. 3.]
[3. 3. 3.]
[3. 3. 3.]
[3. 3. 3.]
[3. 3. 3.]


In [8]:
net.initialize(init=init.Normal(sigma=0.2),device=device,force_reinit=True)
print(net[0].weight.data()[0])
print(net[0].weight.data()[1])
print(net[0].weight.data()[2])
print(net[0].weight.data()[3])
print(net[0].weight.data()[4])

[-0.16771524 -0.22150318 -0.55250967]
[ 0.3559865  -0.09566084 -0.08080624]
[ 0.32348275 -0.10572262  0.29589763]
[-0.25455818 -0.29521954 -0.03771491]
[-0.16730648 -0.11865775  0.25754797]
